In [10]:
import pandas as pd
import numpy as np

In [11]:
"""
src: http://groups.mrl.uiuc.edu/chiang/czoschke/diffraction-selection-rules.html


Allowed planes
primitive - any
Body Centered - h+k+l = 2n even
face centered - h,k,l all odd or all even
fcc diamond - h,k,l all odd || h+k+l = 4n
HCP - L even, H+2K != 3n
"""
def getAllPlanes(n):
    df = pd.DataFrame(columns=["h", "k", "l"])
    for h in range(-n, n+1): 
        for k in range(-n, n+1): 
            for l in range(-n, n+1): 
                i = len(df)
                df.loc[i, "h"] = h
                df.loc[i, "k"] = k
                df.loc[i, "l"] = l
    return df

def isBodyCentered(x): 
    buff = sum(x)
    if buff % 2 == 0: 
        return True
    else:
        return False

def isFaceCentered(x):
    h, k, l = x
    if (h%2 == 1) and (k%2 == 1) and (l%2 == 1): 
        return True
    elif (h%2 == 0) and (k%2 == 0) and (l%2 == 0): 
        return True
    else:
        return False
    
def isFCCDiamond(x): 
    if isFaceCentered(x):
        return True
    if sum(x) % 8 == 1: 
        return True
    return False

def isHCP(x):
    h, k, l  = x
    if l%2 == 0: 
        return True
    buff = h  + 2 * k 
    if buff % 3 == 1: 
        return True
    return False

class dspace:
    def hcp(x, a, c): 
        h, k, l = x
        return 4/3 * (h ** 2 + h * k + k ** 2 ) / a ** 2 + l ** 2 / c ** 2
    def fcc(x, a): 
        h, k, l = x
        return (h **2 + k **2 + l ** 2 ) / a **2
    def tetragonal(x, a, c): 
        h, k, l = x
        return (h ** 2 + k ** 2) / a ** 2 + l ** 2 /  c** 2

In [16]:
"""
For Ba2Y(CuO2)3 
"""
df = getAllPlanes(5)
# df = df[df[["h", "k", "l"]].apply(isBodyCentered, axis=1)]
a = .3891 # nm
c = 1.2123
l = .1542 # nm Cu K alpha radiation
df["1/d2"] = df[["h", "k", "l"]].apply(lambda x: dspace.tetragonal(x, a, c), axis=1)
df["d"] = 1/ df["1/d2"]  ** .5
df["sinq"] = l / (2 *df["d"])
df["q_rad"] = np.arcsin(df["sinq"])
df["q_deg"] =  df["q_rad"] * 360/ 2/ np.pi
df["2q"] = df["q_deg"] * 2
df["sinq/l(A)"] = df["sinq"] / (l * 10)

/home/rose/.local/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in arcsin
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [17]:
"""
Scattering Factor of Each eleemnt
""" 
def scattering_factor(s, z, abPairs):
    """
    s = sinq /l (A)
    z = Z
    ab Pairs: List of tuples 
    """
    tot = 41.78214 * s ** 2 
    summation = 0
    for pair in abPairs:
        a, b = pair
        summation += a * np.exp(-b * s **2)
    return z - tot * summation



# for Ba
z = 56
abPairs = [(7.821, 117.657), (6.004, 18.778), (3.280, 3.263), (1.103, .376)]
df["f_Ba"] = df[["sinq/l(A)"]].apply(lambda x: scattering_factor(x, z, abPairs))
# for Y
z = 39
abPairs = [(4.129, 27.548), (3.012, 5.088), (1.179, .591)]
df["f_Y"] = df[["sinq/l(A)"]].apply(lambda x: scattering_factor(x, z, abPairs))
# for Cu
z =29
abPairs = [(1.579, 62.940), (1.820, 12.453), (1.658, 2.504), (.532, .333)]
df["f_Cu"] = df[["sinq/l(A)"]].apply(lambda x: scattering_factor(x, z, abPairs))
# for O
z = 8
abPairs = [(.455, 23.780), (.917, 7.622), (.472, 2.144), (.138, .296)]
df["f_O"] = df[["sinq/l(A)"]].apply(lambda x: scattering_factor(x, z, abPairs))


In [18]:
data = {
"Ba": [(1/2, 1/2, 0.19711400), (1/2,1/2,0.80288600)],
"Y": [(1/2, 1/2,  1/2)],
"Cu": [(0, 0,0.36812500),(0, 0, 0.63187500 ),(0, 0, 0)],
"O": [(0, 1/2, 0.62090900) , (1/2, 0,0.37909100), (1/2, 0, 0.62090900), (0, 1/2, 0.37909100), (0 , 0, 0.14934000), (0 , 0,0.85066000)]
}

In [19]:
def structureFactor(x, data): 
    def expTerm(coords, h, k, l): 
        tot = 0
        for coord in coords: 
            a,b,c = coord
            tot += np.exp(2 * np.pi * 1j * (a *h + b * k + c * l))
        tot = complex(tot)
        if abs(tot.imag) <= 1e-10: 
            tot = tot.real
        if abs(tot.real) <= 1e-10: 
            tot = tot.imag
        return tot
    
    
    h, k, l, f_ba, f_y, f_cu, f_o = x 

    baBuff = f_ba * expTerm(data["Ba"], h, k, l) 
    yBuff = f_y * expTerm(data["Y"], h, k, l)
    cuBuff = f_cu * expTerm(data["Cu"], h, k, l)
    oBuff = f_o * expTerm(data["O"], h, k, l)
    return baBuff + yBuff + cuBuff + oBuff
    
    
    
    
    
    

In [20]:
# If computations cant be made its a forbidden reflection
df = df.replace(np.inf, np.nan)
df = df.dropna()
# Stucture Factors
df["F"] = df[["h", "k", "l", "f_Ba", "f_Y", "f_Cu", "f_O"]].apply(lambda x: structureFactor(x, data), axis=1) 
df["F2"] = df["F"] ** 2

In [21]:
# Getting counts of each family
df["p"] = df.groupby("d")["d"].transform("count")

In [22]:
df["(1+cos2(2theta))/(sin2(theta))cos(theta))"]  = (1 + np.cos(df["q_rad"] ** 2) ** 2) / (np.sin(df["q_rad"]) ** 2 * np.cos(df["q_rad"]))
df["Intensity"] = df["F2"] * df["p"] * df["(1+cos2(2theta))/(sin2(theta))cos(theta))"]

In [23]:
df = df.sort_values("d")
df.to_csv("Ba2Y(CuO2).csv", index=False)


# Remove Duplicates
df["c"] = df[["h", "k", "l"]].apply(lambda x: x[0] ** 2 + x[1] **2 + x[2] ** 2 , axis =1 )
df["s"] =  df[["h", "k", "l"]].apply(lambda x: x[0]  + x[1] + x[2] , axis =1 )
df = df.sort_values("s",ascending = False)
df = df.drop_duplicates("c", keep="first")
df = df.drop(columns=["s", "c"])

df.to_csv("Ba2Y(CuO2)_Clean.csv", index=False)

In [142]:
df = pd.read_csv("nyloncoords.csv", names=range(12))

In [161]:
buff = {}
l = []
for i in  range(8): 
    d = df[range(9, 12)]
    if i < 6: 
        l.append(tuple(d.loc[i, :].to_list()))
        buff["C"] = l
    elif i == 6: 
        buff["N"] = tuple(d.loc[i, :].to_list())
    else : 
        buff["O"] = tuple(d.loc[i, :].to_list())

In [4]:
"""
Scattering Factor of Each eleemnt
""" 
import numpy as np
def scattering_factor( z, abPairs):
    """
    z = Z
    ab Pairs: List of tuples 
    """
    summation = ""
    for pair in abPairs:
        a, b = pair
        summation += f"+ {a} * exp(- {b} * J149 ^ 2)"
    return f"{z} - 41.78214 * J149 * ({summation})"



# for Ba
z = 56
abPairs = [(7.821, 117.657), (6.004, 18.778), (3.280, 3.263), (1.103, .376)]
print(scattering_factor(z, abPairs))
# for Y
z = 39
abPairs = [(4.129, 27.548), (3.012, 5.088), (1.179, .591)]
# for Cu
z =29
abPairs = [(1.579, 62.940), (1.820, 12.453), (1.658, 2.504), (.532, .333)]
# for O
z = 8
abPairs = [(.455, 23.780), (.917, 7.622), (.472, 2.144), (.138, .296)]

56 - 41.78214 * J149 * (+ 7.821 * exp(- 117.657 * J149 ^ 2)+ 6.004 * exp(- 18.778 * J149 ^ 2)+ 3.28 * exp(- 3.263 * J149 ^ 2)+ 1.103 * exp(- 0.376 * J149 ^ 2))


In [28]:
# From  excel to structure factor 
content  = """
0	0	1	0.6804	1.2123	0.063598119	0.06364107	3.6464	7.2927	0.0412	54.8215	38.4240	28.6174	7.8614
0	0	2	2.7217	0.6062	0.127196239	0.127541742	7.3076	14.6152	0.0825	52.2745	36.8658	27.6178	7.4710
0	0	3	6.1238	0.4041	0.190794358	0.191971306	10.9991	21.9983	0.1237	49.5957	34.7380	26.2933	6.8957
1	0	0	6.6051	0.3891	0.198149576	0.199469702	11.4288	22.8575	0.1285	49.2882	34.4763	26.1298	6.8211
-1	0	-1	7.2855	0.3705	0.208105683	0.20963784	12.0114	24.0227	0.1350	48.8705	34.1206	25.9063	6.7182
1	0	2	9.3268	0.3274	0.235461542	0.237693435	13.6188	27.2377	0.1527	47.7078	33.1448	25.2812	6.4266
1	0	3	12.7289	0.2803	0.275074065	0.278666739	15.9664	31.9329	0.1784	45.9821	31.7707	24.3530	5.9916
1	1	0	13.2101	0.2751	0.280225818	0.284029344	16.2737	32.5474	0.1817	45.7555	31.5974	24.2306	5.9346
1	1	1	13.8906	0.2683	0.287352101	0.291461195	16.6995	33.3990	0.1864	45.4421	31.3600	24.0605	5.8559
1	1	2	15.9318	0.2505	0.307742412	0.312819381	17.9232	35.8465	0.1996	44.5489	30.6970	23.5704	5.6317
0	0	5	17.0106	0.2425	0.317990596	0.323609318	18.5414	37.0829	0.2062	44.1046	30.3729	23.3223	5.5199
1	0	4	17.4919	0.2391	0.322457419	0.328324435	18.8116	37.6232	0.2091	43.9123	30.2336	23.2138	5.4715
1	1	3	19.3340	0.2274	0.339011793	0.345866289	19.8167	39.6334	0.2199	43.2092	29.7274	22.8105	5.2935
1	0	5	23.6157	0.2058	0.374674891	0.384046098	22.0042	44.0084	0.2430	41.7594	28.6865	21.9383	4.9216
1	1	4	24.09693162	0.2037	0.378473303	0.388146348	22.2391	44.4783	0.2454	41.6110	28.5793	21.8454	4.8830
0	0	6	24.49527336	0.2021	0.381588716	0.391514459	22.4321	44.8643	0.2475	41.4902	28.4918	21.7693	4.8516
2	0	0	26.42028692	0.1946	0.396299152	0.407482431	23.3470	46.6940	0.2570	40.9311	28.0840	21.4108	4.7052
2	0	1	27.10071118	0.1921	0.401369828	0.413011939	23.6638	47.3277	0.2603	40.7427	27.9453	21.2877	4.6556
"""
data = {
"Ba": [(1/2, 1/2, 0.19711400), (1/2,1/2,0.80288600)],
"Y": [(1/2, 1/2,  1/2)],
"Cu": [(0, 0,0.36812500),(0, 0, 0.63187500 ),(0, 0, 0)],
"O": [(0, 1/2, 0.62090900) , (1/2, 0,0.37909100), (1/2, 0, 0.62090900), (0, 1/2, 0.37909100), (0 , 0, 0.14934000), (0 , 0,0.85066000)]
}

In [41]:
def structureFactor(x, data): 
    def expTerm(coords, h, k, l): 
        """
        (exp(first coords) + exp (second coord) .... exp(last term ))
        
        :param coords: [(x, y, z), (x, y, z) .... (x, y, z)] for given element  
        :param h, k, l: (int)
        """
        tot = 0
        # Iterates over elements coords and computes the exponent term, adds to total
        for coord in coords: 
            a,b,c = coord
            tot += np.exp(2 * np.pi * 1j * (a *h + b * k + c * l))
            
        # Checking if one term is just super small
        tot = complex(tot)
        if abs(tot.imag) <= 1e-10: 
            tot = tot.real
        if abs(tot.real) <= 1e-10: 
            tot = tot.imag
        return tot
    h, k, l, f_ba, f_y, f_cu, f_o = x 

    # Multiplies Scattering Factor by the exponent term 
    # f_{element} = (exp(first coords) + exp (second coord) .... exp(last term ))
    ba = f_ba * expTerm(data["Ba"], h, k, l) 
    y = f_y * expTerm(data["Y"], h, k, l)
    cu = f_cu * expTerm(data["Cu"], h, k, l)
    o = f_o *expTerm(data["O"], h, k, l)
    
    return ba + y + cu + o

In [42]:
from io import StringIO

name = ['h', 'k', 'l', '1/d2', 'd', 'sin q', 'q rad', 'q deg', ' 2q', 'sinq/l', 'Ba', 'Y', 'Cu', "O"]
df = pd.read_csv(StringIO(content), sep="\t", names = name)

df["F"] = df[["h", "k", "l", "Ba", "Y", "Cu", "O"]].apply(lambda x: structureFactor(x, data), axis=1)


In [48]:
buff = []
for h, k , l in zip(df.h, df.k , df.l):
    buff.append((h, k ,l))
print(buff)

[(0, 0, 1), (0, 0, 2), (0, 0, 3), (1, 0, 0), (-1, 0, -1), (1, 0, 2), (1, 0, 3), (1, 1, 0), (1, 1, 1), (1, 1, 2), (0, 0, 5), (1, 0, 4), (1, 1, 3), (1, 0, 5), (1, 1, 4), (0, 0, 6), (2, 0, 0), (2, 0, 1)]


In [50]:
# For spot diffraction
# From  excel to structure factor 
content  = """
0	0	1
0	0	-1
		
0	0	2
0	0	-2
		
0	0	3
0	0	-3
		
-1	0	0
0	1	0
0	-1	0
1	0	0
		
-1	0	-1
-1	0	1
0	-1	1
0	-1	-1
0	1	1
0	1	-1
1	0	-1
1	0	1
		
-1	0	-2
-1	0	2
0	-1	2
0	1	-2
0	1	2
0	-1	-2
1	0	-2
1	0	2
		
0	0	4
0	0	-4
		
-1	0	3
-1	0	-3
0	-1	3
0	-1	-3
0	1	-3
0	1	3
1	0	-3
1	0	3
		
-1	-1	0
-1	1	0
1	1	0
1	-1	0
		
-1	-1	-1
-1	-1	1
-1	1	-1
-1	1	1
1	-1	1
1	1	-1
1	1	1
1	-1	-1
		
-1	1	-2
-1	-1	-2
-1	1	2
-1	-1	2
1	1	2
1	1	-2
1	-1	-2
1	-1	2
		
0	0	-5
0	0	5
		
-1	0	4
-1	0	-4
0	-1	-4
0	1	4
0	1	-4
0	-1	4
1	0	-4
1	0	4
		
-1	1	3
-1	-1	3
-1	-1	-3
-1	1	-3
1	-1	-3
1	1	-3
1	1	3
1	-1	3
		
-1	0	5
-1	0	-5
0	-1	-5
0	-1	5
0	1	-5
0	1	5
1	0	5
1	0	-5
		
-1	-1	-4
-1	1	4
-1	1	-4
-1	-1	4
1	1	-4
1	-1	-4
1	-1	4
1	1	4
		
-2	0	0
0	2	0
0	-2	0
2	0	0
		
-2	0	-1
-2	0	1
0	-2	-1
0	-2	1
0	2	-1
0	2	1
2	0	1
2	0	-1
		
-2	0	2
-2	0	-2
0	-2	2
0	2	2
0	-2	-2
0	2	-2
2	0	2
2	0	-2
		
-1	-1	5
-1	1	-5
-1	1	5
-1	-1	-5
1	-1	-5
1	-1	5
1	1	5
1	1	-5
		
-2	0	3
-2	0	-3
0	2	-3
0	-2	3
0	-2	-3
0	2	3
2	0	-3
2	0	3
		
-2	1	0
-2	-1	0
-1	-2	0
-1	2	0
1	-2	0
1	2	0
2	-1	0
2	1	0
		
-2	1	-1
-2	1	1
-2	-1	1
-2	-1	-1
-1	2	1
-1	-2	1
-1	2	-1
-1	-2	-1
1	-2	1
1	2	-1
1	2	1
1	-2	-1
2	1	-1
2	-1	-1
2	-1	1
2	1	1
		
-2	1	-2
-2	-1	2
-2	-1	-2
-2	1	2
-1	2	-2
-1	-2	-2
-1	-2	2
-1	2	2
1	-2	2
1	2	2
1	2	-2
1	-2	-2
2	-1	-2
2	-1	2
2	1	2
2	1	-2
		
-2	0	-4
-2	0	4
0	2	-4
0	2	4
0	-2	4
0	-2	-4
2	0	-4
2	0	4
		
-2	-1	3
-2	-1	-3
-2	1	3
-2	1	-3
-1	-2	3
-1	-2	-3
-1	2	-3
-1	2	3
1	-2	-3
1	-2	3
1	2	-3
1	2	3
2	1	3
2	-1	-3
2	1	-3
2	-1	3
		
-2	0	-5
-2	0	5
0	2	-5
0	-2	5
0	2	5
0	-2	-5
2	0	-5
2	0	5
		
-2	-1	4
-2	-1	-4
-2	1	4
-2	1	-4
-1	2	-4
-1	-2	4
-1	-2	-4
-1	2	4
1	-2	4
1	-2	-4
1	2	4
1	2	-4
2	1	4
2	1	-4
2	-1	4
2	-1	-4
		
-2	-1	-5
-2	1	-5
-2	-1	5
-2	1	5
-1	2	5
-1	-2	-5
-1	-2	5
-1	2	-5
1	2	-5
1	2	5
1	-2	-5
1	-2	5
2	-1	-5
2	-1	5
2	1	-5
2	1	5
		
-2	-2	0
-2	2	0
2	2	0
2	-2	0
		
-2	-2	-1
-2	-2	1
-2	2	1
-2	2	-1
2	-2	1
2	-2	-1
2	2	-1
2	2	1
		
-2	-2	2
-2	2	2
-2	-2	-2
-2	2	-2
2	2	2
2	-2	2
2	-2	-2
2	2	-2
		
-2	2	3
-2	2	-3
-2	-2	-3
-2	-2	3
2	2	-3
2	-2	-3
2	2	3
2	-2	3
		
-3	0	0
0	3	0
0	-3	0
3	0	0
		
-3	0	1
-3	0	-1
0	3	1
0	-3	1
0	3	-1
0	-3	-1
3	0	1
3	0	-1
		
-3	0	-2
-3	0	2
0	3	2
0	3	-2
0	-3	2
0	-3	-2
3	0	-2
3	0	2
		
-2	2	4
-2	2	-4
-2	-2	4
-2	-2	-4
2	-2	4
2	2	-4
2	2	4
2	-2	-4
		
-3	0	3
-3	0	-3
0	-3	-3
0	3	3
0	-3	3
0	3	-3
3	0	-3
3	0	3
		
-3	1	0
-3	-1	0
-1	-3	0
-1	3	0
1	3	0
1	-3	0
3	-1	0
3	1	0
		
-3	-1	-1
-3	1	-1
-3	1	1
-3	-1	1
-1	3	-1
-1	3	1
-1	-3	1
-1	-3	-1
1	3	1
1	3	-1
1	-3	-1
1	-3	1
3	-1	-1
3	-1	1
3	1	-1
3	1	1
		
-3	1	2
-3	-1	2
-3	1	-2
-3	-1	-2
-1	3	-2
-1	3	2
-1	-3	2
-1	-3	-2
1	3	-2
1	3	2
1	-3	2
1	-3	-2
3	-1	-2
3	1	2
3	-1	2
3	1	-2
		
-2	2	-5
-2	-2	-5
-2	2	5
-2	-2	5
2	2	-5
2	-2	-5
2	-2	5
2	2	5
		
-3	0	-4
-3	0	4
0	-3	4
0	-3	-4
0	3	4
0	3	-4
3	0	-4
3	0	4
		
-3	-1	3
-3	-1	-3
-3	1	-3
-3	1	3
-1	-3	3
-1	3	-3
-1	3	3
-1	-3	-3
1	3	3
1	3	-3
1	-3	3
1	-3	-3
3	-1	-3
3	-1	3
3	1	3
3	1	-3
		
-3	0	-5
-3	0	5
0	-3	-5
0	-3	5
0	3	5
0	3	-5
3	0	-5
3	0	5
		
-3	-1	-4
-3	1	-4
-3	1	4
-3	-1	4
-1	3	4
-1	-3	-4
-1	3	-4
-1	-3	4
1	-3	-4
1	3	4
1	-3	4
1	3	-4
3	1	-4
3	-1	4
3	1	4
3	-1	-4
		
-3	1	-5
-3	-1	5
-3	-1	-5
-3	1	5
-1	3	-5
-1	-3	5
-1	-3	-5
-1	3	5
1	-3	5
1	-3	-5
1	3	-5
1	3	5
3	-1	5
3	1	5
3	-1	-5
3	1	-5
		
-3	-2	0
-3	2	0
-2	3	0
-2	-3	0
2	-3	0
2	3	0
3	2	0
3	-2	0
		
-3	2	-1
-3	-2	1
-3	-2	-1
-3	2	1
-2	3	1
-2	-3	1
-2	-3	-1
-2	3	-1
2	-3	-1
2	-3	1
2	3	-1
2	3	1
3	-2	1
3	-2	-1
3	2	-1
3	2	1
		
-3	-2	2
-3	-2	-2
-3	2	2
-3	2	-2
-2	-3	2
-2	3	2
-2	-3	-2
-2	3	-2
2	-3	2
2	-3	-2
2	3	-2
2	3	2
3	-2	2
3	2	2
3	-2	-2
3	2	-2
		
-3	2	3
-3	2	-3
-3	-2	3
-3	-2	-3
-2	-3	-3
-2	3	-3
-2	3	3
-2	-3	3
2	-3	-3
2	3	-3
2	3	3
2	-3	3
3	-2	-3
3	-2	3
3	2	-3
3	2	3
		
-3	2	4
-3	2	-4
-3	-2	-4
-3	-2	4
-2	-3	4
-2	3	-4
-2	-3	-4
-2	3	4
2	3	4
2	-3	-4
2	3	-4
2	-3	4
3	2	-4
3	2	4
3	-2	4
3	-2	-4
		
-3	-2	5
-3	-2	-5
-3	2	-5
-3	2	5
-2	-3	-5
-2	-3	5
-2	3	-5
-2	3	5
2	3	5
2	-3	5
2	3	-5
2	-3	-5
3	-2	-5
3	2	-5
3	2	5
3	-2	5
		
-4	0	0
0	-4	0
0	4	0
4	0	0
		
-4	0	-1
-4	0	1
0	-4	1
0	-4	-1
0	4	-1
0	4	1
4	0	1
4	0	-1
		
-4	0	-2
-4	0	2
0	4	-2
0	-4	2
0	4	2
0	-4	-2
4	0	2
4	0	-2
		
-4	0	3
-4	0	-3
0	-4	3
0	-4	-3
0	4	-3
0	4	3
4	0	3
4	0	-3
		
-4	1	0
-4	-1	0
-1	4	0
-1	-4	0
1	-4	0
1	4	0
4	-1	0
4	1	0
		
-4	-1	1
-4	1	-1
-4	1	1
-4	-1	-1
-1	-4	-1
-1	4	-1
-1	4	1
-1	-4	1
1	-4	1
1	4	-1
1	4	1
1	-4	-1
4	1	1
4	1	-1
4	-1	1
4	-1	-1
		
-4	1	-2
-4	-1	2
-4	1	2
-4	-1	-2
-1	-4	2
-1	-4	-2
-1	4	2
-1	4	-2
1	4	-2
1	4	2
1	-4	-2
1	-4	2
4	-1	2
4	-1	-2
4	1	2
4	1	-2
		
-4	0	-4
-4	0	4
0	4	-4
0	4	4
0	-4	-4
0	-4	4
4	0	-4
4	0	4
		
-4	1	-3
-4	-1	-3
-4	-1	3
-4	1	3
-1	-4	-3
-1	4	-3
-1	4	3
-1	-4	3
1	-4	3
1	4	-3
1	4	3
1	-4	-3
4	-1	3
4	1	3
4	1	-3
4	-1	-3
		
-3	-3	0
-3	3	0
3	3	0
3	-3	0
		
-3	-3	-1
-3	3	-1
-3	3	1
-3	-3	1
3	-3	1
3	-3	-1
3	3	1
3	3	-1
		
-3	3	2
-3	-3	-2
-3	-3	2
-3	3	-2
3	3	2
3	-3	-2
3	3	-2
3	-3	2
		
-4	0	5
-4	0	-5
0	4	5
0	-4	-5
0	-4	5
0	4	-5
4	0	-5
4	0	5
		
-4	1	4
-4	1	-4
-4	-1	4
-4	-1	-4
-1	-4	4
-1	-4	-4
-1	4	4
-1	4	-4
1	4	4
1	4	-4
1	-4	4
1	-4	-4
4	-1	4
4	-1	-4
4	1	4
4	1	-4
		
-3	-3	-3
-3	3	3
-3	3	-3
-3	-3	3
3	-3	-3
3	3	-3
3	3	3
3	-3	3
		
-4	-1	-5
-4	1	-5
-4	1	5
-4	-1	5
-1	-4	-5
-1	-4	5
-1	4	-5
-1	4	5
1	4	5
1	4	-5
1	-4	5
1	-4	-5
4	1	-5
4	-1	-5
4	-1	5
4	1	5
		
-3	-3	-4
-3	3	-4
-3	3	4
-3	-3	4
3	3	4
3	3	-4
3	-3	4
3	-3	-4
		
-4	-2	0
-4	2	0
-2	-4	0
-2	4	0
2	4	0
2	-4	0
4	2	0
4	-2	0
		
-4	-2	-1
-4	-2	1
-4	2	1
-4	2	-1
-2	4	1
-2	-4	1
-2	-4	-1
-2	4	-1
2	-4	-1
2	4	-1
2	-4	1
2	4	1
4	2	-1
4	2	1
4	-2	-1
4	-2	1
		
-4	-2	-2
-4	2	-2
-4	2	2
-4	-2	2
-2	4	-2
-2	4	2
-2	-4	2
-2	-4	-2
2	4	-2
2	-4	-2
2	4	2
2	-4	2
4	2	-2
4	-2	2
4	-2	-2
4	2	2
		
-3	-3	5
-3	3	-5
-3	-3	-5
-3	3	5
3	3	5
3	-3	-5
3	-3	5
3	3	-5
		
-4	-2	-3
-4	2	3
-4	-2	3
-4	2	-3
-2	4	-3
-2	4	3
-2	-4	-3
-2	-4	3
2	-4	-3
2	-4	3
2	4	3
2	4	-3
4	2	-3
4	-2	3
4	-2	-3
4	2	3
		
-4	2	-4
-4	-2	-4
-4	-2	4
-4	2	4
-2	-4	-4
-2	4	4
-2	4	-4
-2	-4	4
2	4	-4
2	-4	4
2	-4	-4
2	4	4
4	-2	-4
4	2	4
4	-2	4
4	2	-4
		
-4	-2	-5
-4	-2	5
-4	2	5
-4	2	-5
-2	4	5
-2	-4	-5
-2	-4	5
-2	4	-5
2	4	-5
2	-4	5
2	4	5
2	-4	-5
4	2	5
4	-2	-5
4	-2	5
4	2	-5
		
-5	0	0
-4	3	0
-4	-3	0
-3	4	0
-3	-4	0
0	-5	0
0	5	0
3	4	0
3	-4	0
4	3	0
4	-3	0
5	0	0
		
-5	0	1
-5	0	-1
-4	3	1
-4	-3	-1
-4	-3	1
-4	3	-1
-3	-4	-1
-3	-4	1
-3	4	1
-3	4	-1
0	5	1
0	5	-1
0	-5	1
0	-5	-1
3	-4	-1
3	4	-1
3	4	1
3	-4	1
4	-3	1
4	-3	-1
4	3	-1
4	3	1
5	0	1
5	0	-1
		
-5	0	-2
-5	0	2
-4	3	2
-4	3	-2
-4	-3	2
-4	-3	-2
-3	-4	-2
-3	-4	2
-3	4	-2
-3	4	2
0	-5	-2
0	-5	2
0	5	-2
0	5	2
3	-4	-2
3	-4	2
3	4	-2
3	4	2
4	-3	-2
4	-3	2
4	3	-2
4	3	2
5	0	-2
5	0	2
"""
data = {
"Ba": [(1/2, 1/2, 0.19711400), (1/2,1/2,0.80288600)],
"Y": [(1/2, 1/2,  1/2)],
"Cu": [(0, 0,0.36812500),(0, 0, 0.63187500 ),(0, 0, 0)],
"O": [(0, 1/2, 0.62090900) , (1/2, 0,0.37909100), (1/2, 0, 0.62090900), (0, 1/2, 0.37909100), (0 , 0, 0.14934000), (0 , 0,0.85066000)]
}

In [63]:
def expTerm(x, coords): 
    """
    (exp(first coords) + exp (second coord) .... exp(last term ))

    :param coords: [(x, y, z), (x, y, z) .... (x, y, z)] for given element  
    :param h, k, l: (int)
    """
    h, k, l = x
    tot = 0
    # Iterates over elements coords and computes the exponent term, adds to total
    for coord in coords: 
        a,b,c = coord
        tot += np.exp(2 * np.pi * 1j * (a *h + b * k + c * l))

    # Checking if one term is just super small
    tot = complex(tot)
    if abs(tot.imag) <= 1e-10: 
        tot = tot.real
    if abs(tot.real) <= 1e-10: 
        tot = tot.imag
    return tot

data = {
"Ba": [(1/2, 1/2, 0.19711400), (1/2,1/2,0.80288600)],
"Y": [(1/2, 1/2,  1/2)],
"Cu": [(0, 0,0.36812500),(0, 0, 0.63187500 ),(0, 0, 0)],
"O": [(0, 1/2, 0.62090900) , (1/2, 0,0.37909100), (1/2, 0, 0.62090900), (0, 1/2, 0.37909100), (0 , 0, 0.14934000), (0 , 0,0.85066000)]
}

name = ['h', 'k', 'l']
df = pd.read_csv(StringIO(content), sep="\t", names = name)


for k in data.keys(): 
    df[k] = df[["h", "k", "l"]].apply(lambda x: expTerm(x, data[k]), axis=1)
    
def isForbidden(x): 
    for val in x: 
        if abs(val.real) > .1 or abs(val.imag) > .1: 
            return ""
        elif np.isnan(val): 
            return ""
    return "forbidden"
        
df["forbidden"] = df[data.keys()].apply(lambda x: isForbidden(x) , axis=1)

In [64]:
df.to_csv("temp.csv", index=False)